In [1]:
import networkx as nx

from OperatorBenchmark import OperatorBenchmark

from typing import List, Dict

from scripts import load_profiling_data,create_profiling_data_ER,separate_keys_and_values,git_branch_commit_push,create_profiling_data_ER_HPC

from scripts_old import load_runMultipleSimpleQWAK, load_runMultipleSimpleQWAK_legacy
from datetime import datetime

import os
import re

from utils.plotTools import plot_qwak

import numpy as np
import cupy as cp
from scipy.linalg import inv, expm
import networkx as nx
import time
import cupyx.scipy.linalg as cpx_scipy
from cupyx.profiler import benchmark
from matplotlib import pyplot as plt
import os
import json
import pickle
from tqdm import tqdm
import subprocess
import random
from qwak_cupy.qwak import QWAK as CQWAK
from qwak.qwak import QWAK as QWAK
from datetime import datetime

Profiling applied to init_operator, config: {'output_path': 'operator_results', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to build_operator, config: {'output_path': 'operator_results', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to build_expm_operator, config: {'output_path': 'operator_results_hpc', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to init_operator, config: {'output_path': 'operator_results_hpc', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to build_operator, config: {'output_path': 'operator_results_hpc', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to build_expm_operator, config: {'output_path': 'operator_results_hpc', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to init_hiperwal

In [12]:

nMin = 3
nMax = 1000 
n_values = list(range(nMin, nMax, 1))
pVal = 0.8
sample_range = range(0, 100, 1)
seed = 10


SCRIPT_DIR = os.getcwd()
path = os.path.normpath(os.path.join(
    SCRIPT_DIR,
    "Profiling/operator_results"
))
# Verify path exists
if not os.path.exists(path):
    print(f"ERROR: Path not found - {path}")
    print("Current directory contents:", os.listdir(SCRIPT_DIR))
    exit()
# Now load data
result = load_profiling_data(
    path=path,
    method_name="init_operator",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)
result2 = load_profiling_data(
    path=path,
    method_name="build_operator",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)

result3 = load_profiling_data(
    path=path,
    method_name="build_expm_operator",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)

Processing n-values: 100%|██████████| 997/997 [00:51<00:00, 19.29it/s]


In [13]:
def merge_by_sum(dict_a, dict_b):
    """
    Merge two dictionaries by summing the values of matching keys.
    Assumes both dictionaries have the same keys.
    """
    merged = {}
    for key in dict_a:
        merged[key] = dict_a[key] + dict_b[key]
    return merged


params = {
    'figsize': (12, 8),
    'plot_title' : f'Spectral Decomposition vs Expm',
    'x_label' : 'Graph Size N',
    'y_label' : "Time",
    'legend_labels' : ['spec decomp','expm'],
    'legend_loc': "best",
    'legend_title' : 'Solutions',
    'legend_ncol' : 1,
    # 'color_list' : ['#0000FF', '#008000', '#525252'],
    'color_list' : ['b','g','r'],
    'line_style_list' : ['--', '-','-.' ],
    'save_path' : f'ImgOutput/benchmark-OPERATOR.png',
    'use_loglog': False,
    'use_cbar' : False,
    'cbar_label' : None, 
    'cbar_ticks' : None,
    'cbar_tick_labels' : None,
    'x_lim' : None,
    # 'x_num_ticks' : 10,
    'y_num_ticks' : 5,
    'x_round_val' : 2,
    'y_round_val' : 3,
    # 'v_line_values' : v_line_values,
    # # 'v_line_style': '--',
    'title_font_size': 20,
    'xlabel_font_size': 22,
    'ylabel_font_size': 22,
    'legend_font_size': 14,
    'legend_title_font_size': 14,
    'tick_font_size': 18,}

merged_result = merge_by_sum(result, result2)
result_keys, result_values = separate_keys_and_values(merged_result)
# result_keys, result_values = separate_keys_and_values(result)
# result2_keys, result2_values = separate_keys_and_values(result2)
result3_keys, result3_values = separate_keys_and_values(result3)



x_value_matrix = [result_keys,result3_keys]
y_value_matrix = [result_values,result3_values]

plot_qwak(x_value_matrix = x_value_matrix, y_value_matrix = y_value_matrix,**params)

# QWAK Results - Old Method

In [33]:


# Parameters
nMin = 3
nMax = 1000
nList = list(range(nMin, nMax, 1))
pVal = 0.8
samples = 100
t = 100

# base_dir = 'Datasets/Benchmark-SimpleQWAK-Test_ER2'
# base_dir = 'Datasets/Benchmark-SimpleQWAK-Test_ER2'
# base_dir_cupy = 'Datasets/Benchmark-SimpleQWAK_ER3_CuPy'

base_dir_cupy_970 = 'Profiling\Old\Benchmark-SimpleQWAK_ER-CuPy_970'
base_dir_cupy_3070 = 'Profiling\Old\Benchmark-SimpleQWAK_ER2_CuPy_3070'
base_dir_cupy_3070_2 = 'Profiling\Old\Benchmark-SimpleQWAK_ER2_CuPy_3070_2'
base_dir = 'Profiling\Old\Benchmark-SimpleQWAK_ER-NumPy'


# # Record start datetime
start_datetime = datetime.now()

# runMultipleSimpleQWAK(nList, pVal, t, samples, base_dir, hpc = False)
avg_list = load_runMultipleSimpleQWAK_legacy(nList, pVal,samples, t, base_dir)

avg_list_cupy_970 = load_runMultipleSimpleQWAK_legacy(nList, pVal,samples, t, base_dir_cupy_970)

avg_list_cupy_3070_2 = load_runMultipleSimpleQWAK_legacy(nList, pVal,samples, t, base_dir_cupy_3070_2)
    
# runMultipleSimpleQWAK(nList, pVal, t, samples, base_dir_cupy_3070, hpc = True)
tList_cupy_3070, qwList_cupy_3070,avg_list_cupy_3070 = load_runMultipleSimpleQWAK(nList, pVal,samples, t, base_dir_cupy_3070)

print('CuPy QWAK results computed and saved.')


# print(avg_list_cupy_3070)
print(avg_list)
print(avg_list_cupy_970)
# Print elapsed time
elapsed_time = datetime.now() - start_datetime
print(f"Elapsed time: {elapsed_time}")

plt.plot(nList,avg_list,label='QWAK CPU_NumPy')
# plt.plot(nList,avg_list_cupy_3070,label='QWAK GPU_CuPy3070')
# plt.plot(nList,avg_list_cupy_3070_2,label='QWAK GPU_CuPy3070_2')
plt.plot(nList,avg_list_cupy_970,label='QWAK GPU_CuPy970')
plt.legend()
plt.show()

Loading NPQWAK data: 100%|██████████| 997/997 [00:00<00:00, 3368.23it/s]


CuPy QWAK results computed and saved.
[0.0003695344924926758, 0.0002491664886474609, 0.0002730965614318848, 0.00029331445693969727, 0.00030520200729370115, 0.0008480048179626465, 0.0003675675392150879, 0.0003970623016357422, 0.0004354667663574219, 0.00046991586685180665, 0.0005152750015258789, 0.0005548691749572754, 0.0005992484092712402, 0.0006510663032531738, 0.0006942009925842285, 0.0007622432708740234, 0.0008188748359680176, 0.0008980917930603027, 0.0009511280059814454, 0.0010181283950805664, 0.0010911917686462402, 0.0011606287956237792, 0.0012433171272277833, 0.0013071417808532715, 0.0013578176498413087, 0.0014470267295837403, 0.00153872013092041, 0.0016238832473754884, 0.0017114734649658202, 0.0018235135078430176, 0.0019228935241699218, 0.0019850969314575196, 0.002095968723297119, 0.0021860289573669436, 0.002304184436798096, 0.0023999738693237303, 0.0025352001190185546, 0.0026423239707946775, 0.002843198776245117, 0.0029297709465026856, 0.003067643642425537, 0.0031784915924072264

In [34]:
# Parameters
nMin = 3
nMax = 1000
nList = list(range(nMin, nMax, 1))
pVal = 0.8
samples = 100
t = 100

base_dir_cupy_970 = 'Profiling\Old\Benchmark-SimpleQWAK_ER-CuPy_970'
base_dir = 'Profiling\Old\Benchmark-SimpleQWAK_ER-NumPy'


avg_list = load_runMultipleSimpleQWAK_legacy(nList, pVal,samples, t, base_dir)

avg_list_cupy_970 = load_runMultipleSimpleQWAK_legacy(nList, pVal,samples, t, base_dir_cupy_970)

params = {
    'figsize': (12, 8),
    'plot_title' : 'Spectral Decomposition vs Expm',
    'x_label' : 'Graph Size N',
    'y_label' : "Time",
    'legend_labels' : ['spec decomp','expm'],
    'legend_loc': "best",
    'legend_title' : 'Solutions',
    'legend_ncol' : 1,
    'color_list' : ['b','g','r'],
    'line_style_list' : ['--', '-','-.'],
    'use_loglog': False,
    'use_cbar' : False,
    'cbar_label' : None, 
    'cbar_ticks' : None,
    'cbar_tick_labels' : None,
    'x_lim' : None,
    'y_num_ticks' : 5,
    'x_round_val' : 2,
    'y_round_val' : 3,
    'title_font_size': 20,
    'xlabel_font_size': 22,
    'ylabel_font_size': 22,
    'legend_font_size': 14,
    'legend_title_font_size': 14,
    'tick_font_size': 18,
}


# ------------------------------------------
# Example usage within your code:
# ------------------------------------------
# Assume you have nList, avg_list, avg_list_cupy_970, and params from before.

# 1) Smooth each data series using LOESS:
x_value_matrix = [nList,nList]
y_value_matrix = [avg_list,avg_list_cupy_970]

plot_qwak(x_value_matrix = x_value_matrix, y_value_matrix = y_value_matrix,**params)

Loading NPQWAK data: 100%|██████████| 997/997 [00:00<00:00, 5086.80it/s]


In [30]:
import numpy as np
import matplotlib.pyplot as plt

# If you don't have statsmodels installed, do:
#   pip install statsmodels
from statsmodels.nonparametric.smoothers_lowess import lowess

def smooth_data_lowess(x, y, frac=0.05):
    """
    Smooth data using a LOESS (Locally Weighted Scatterplot Smoothing) approach.
    LOESS fits simple models to localized subsets of the data to capture
    the overall trend while reducing noise or bumps.

    :param x: 1D array-like of x data
    :param y: 1D array-like of y data
    :param frac: Between 0 and 1; the fraction of data used when estimating each y value.
                 Smaller values of frac create a tighter fit (less smoothing),
                 while larger values create a smoother curve.
    :return: (x_smooth, y_smooth)
    """
    # lowess returns an Nx2 array where:
    #   first column is the sorted x,
    #   second column is the corresponding smoothed y.
    smoothed = lowess(y, x, frac=frac)
    x_smooth = smoothed[:, 0]
    y_smooth = smoothed[:, 1]
    return x_smooth, y_smooth

# ------------------------------------------
# Example usage within your code:
# ------------------------------------------
# Assume you have nList, avg_list, avg_list_cupy_970, and params from before.

# 1) Smooth each data series using LOESS:
x_smooth_0, y_smooth_0 = smooth_data_lowess(nList, avg_list, frac=0.05)
x_smooth_1, y_smooth_1 = smooth_data_lowess(nList, avg_list_cupy_970, frac=0.05)

# 2) Use these smoothed data arrays when calling plot_qwak:
x_value_matrix_smooth = [x_smooth_0, x_smooth_1]
y_value_matrix_smooth = [y_smooth_0, y_smooth_1]

plot_qwak(x_value_matrix=x_value_matrix_smooth,
          y_value_matrix=y_value_matrix_smooth,
          **params)
plt.show()
